In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [153]:
prompt = '''
Please summarize this text:
Thekla (820s or 830s – after 870) was a princess of the Amorian dynasty of the Byzantine Empire. The eldest of seven children of the emperor Theophilos and empress Theodora, she was proclaimed augusta (an imperial title) in the late 830s. After her father's death in 842, her mother became regent for her younger brother Michael III, and Thekla was associated with the regime as a co-empress alongside Theodora and Michael. Thekla was deposed by Michael, possibly alongside her mother, in 856 and consigned to a convent in Constantinople. In one narrative, accepted by some Byzantinists and rejected by others, she became the mistress of Michael's friend and co-emperor Basil I, but was neglected after Basil murdered Michael in 867 and took power as the sole emperor. In this narrative, she took another lover, was discovered, and fell out of favor, then was beaten and had her property confiscated.
'''


In [199]:
import torch
torch.set_grad_enabled(False)
top_p = 0.95
proba_threshold = 0.01
max_steps = 200
top_k = 10
temperature = 1

def get_completions(prompt, top_p=0.9,     stopwords = ['\n','</s>'],proba_threshold=0.01, max_steps=200, temperature=1, top_k=10):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    complete_text = []
    incomplete_text = []
    inputs = model_inputs['input_ids'][0].cpu()
    que = []
    step = 0
    que = [(1, inputs)]
    while step < max_steps and len(que)>0:
        print(step, end=',')
        step+=1
        logproba, item = que.pop()
        
        item = torch.tensor(item)
        result = model(item.unsqueeze(0).cuda())
        logits = result.logits[:,-1,:]/ temperature
        
        arg1=torch.argsort(logits[0]).cpu()
        logits_sorted = logits[:, arg1].cpu()
        probas = torch.softmax(logits[:, arg1], -1).cpu()
        
        i = 1
        total_proba = 0
        while i< len(probas[0]) and i <= top_k:
            total_proba += probas[0,-i]
            if total_proba >= top_p:
                break
            i+=1
        probas_all = torch.softmax(probas[0,-i:],-1)
        logits_all = logits_sorted[0,-i:]
        idx_all = arg1[-i:]   
        for (l, idx) in zip(probas_all, idx_all):
            new_prompt = torch.cat([item, torch.tensor([idx])])
            p = logproba*l.item()
            txt = tokenizer.decode(new_prompt[len(inputs):])
            complete_flag = False
            for stop in stopwords:
                if stop in txt:
                    complete_flag = True
            if complete_flag:
                complete_text.append((p, txt))
                continue
            if p < proba_threshold:
                incomplete_text.append((p, txt))
                continue
            que.append((p, new_prompt))
        que = sorted(que, key = lambda x: -x[0])
    q2=[]
    for q in que:
        p, tokens = q
        q2.append((p, tokenizer.decode(tokens[len(inputs):])))
    return complete_text, incomplete_text, q2

In [190]:
complete, incomplete, que= get_completions(prompt,     stopwords = ['\n','</s>','<|im_end|>'])

0,1,2,3,4,

/tmp/ipykernel_1258/1744391355.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = torch.tensor(item)


5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,

In [192]:
sorted(incomplete,key=lambda x: -x[0])

[(0.009860701788467941, 'Thekla, the eldest of seven'),
 (0.009813518369148523, 'Thekla was the eldest child of Emperor'),
 (0.009710695750769, 'Thekla was a Byzantine princess,'),
 (0.009593654495482874, 'Thekla was the youngest of seven'),
 (0.009589877010184074, 'Thekla was the eldest of seven children of the'),
 (0.009584273629304271, 'Thekla, an imperial princess'),
 (0.009488856179889371, 'Thekla was a Byzantine princess of'),
 (0.009462947032502856, 'Thekla, daughter of The'),
 (0.009411315451612648, 'Thekla, born in'),
 (0.009146838494013756, 'Thekla, daughter of emperor'),
 (0.008958329977186935, 'Thekla was the eldest child of The'),
 (0.00889600259863274, 'Thekla was a princess of the Byzantine Empire.'),
 (0.008787976156174404, 'Thekla was the eldest daughter of The'),
 (0.008765844382549558, 'Thekla was the daughter of Theophilos and'),
 (0.008679106141169072, 'Thekla was an Amorian princess of'),
 (0.008388931553644249, 'Thekla was the eldest daughter of emperor'),
 (0.00

In [173]:
complete, incomplete, que= get_completions("Who is president of Germany?",proba_threshold=0.03)

0,1,2,3,4,5,6,

/tmp/ipykernel_1258/4236127232.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = torch.tensor(item)


7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,

In [193]:
sorted(incomplete,key=lambda x: -x[0])

[(0.009860701788467941, 'Thekla, the eldest of seven'),
 (0.009813518369148523, 'Thekla was the eldest child of Emperor'),
 (0.009710695750769, 'Thekla was a Byzantine princess,'),
 (0.009593654495482874, 'Thekla was the youngest of seven'),
 (0.009589877010184074, 'Thekla was the eldest of seven children of the'),
 (0.009584273629304271, 'Thekla, an imperial princess'),
 (0.009488856179889371, 'Thekla was a Byzantine princess of'),
 (0.009462947032502856, 'Thekla, daughter of The'),
 (0.009411315451612648, 'Thekla, born in'),
 (0.009146838494013756, 'Thekla, daughter of emperor'),
 (0.008958329977186935, 'Thekla was the eldest child of The'),
 (0.00889600259863274, 'Thekla was a princess of the Byzantine Empire.'),
 (0.008787976156174404, 'Thekla was the eldest daughter of The'),
 (0.008765844382549558, 'Thekla was the daughter of Theophilos and'),
 (0.008679106141169072, 'Thekla was an Amorian princess of'),
 (0.008388931553644249, 'Thekla was the eldest daughter of emperor'),
 (0.00

In [200]:
complete, incomplete, que= get_completions("Please paraphrase 'Apple made nice mobile phone called iPhone'", stopwords=['<|im_end'], proba_threshold=0.002, temperature=0.3)

0,1,2,3,4,5,

/tmp/ipykernel_1258/3748198728.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = torch.tensor(item)


6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,

In [202]:
sorted(complete,key=lambda x: -x[0])

[(0.02821499606905706,
  'Apple created a good mobile phone called iPhone.<|im_end|>'),
 (0.026068591505265457,
  'Apple created a nice mobile phone called iPhone.<|im_end|>'),
 (0.023879007952248347,
  'Apple produced a good mobile phone called iPhone.<|im_end|>'),
 (0.02367455328212441,
  'Apple created a pleasant mobile phone known as iPhone.<|im_end|>'),
 (0.023649032726102873,
  'Apple created a delightful mobile phone known as the iPhone.<|im_end|>'),
 (0.021769543856659697,
  'Apple produced a pleasant mobile phone known as iPhone.<|im_end|>'),
 (0.019258433010084147,
  'Apple created a delightful mobile phone known as iPhone.<|im_end|>'),
 (0.01750126859306394,
  'Apple created a well-received mobile phone called iPhone.<|im_end|>'),
 (0.017444559469786326,
  'Apple created a pleasant mobile phone called iPhone.<|im_end|>'),
 (0.01735546409410101,
  'Apple produced a high-quality mobile phone called iPhone.<|im_end|>'),
 (0.01732611543730231,
  'Apple created a popular mobile p